In [2]:
import torch
from torch import nn

# 单隐藏层的多层感知机
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
print(net(X))

tensor([[-0.0800],
        [-0.1247]], grad_fn=<AddmmBackward0>)


In [3]:
# 参数访问
# 通过索引访问模型的任意层
# 检查第二个全连接层参数
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.1571,  0.1150,  0.1987,  0.2166, -0.0739,  0.2368, -0.1162,  0.2392]])), ('bias', tensor([-0.1917]))])


In [5]:
# 目标参数
# 提取第二个全连接层的偏置
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
# 没有调用反向传播之前，参数的梯度是None
print(net[2].bias.grad == None)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1917], requires_grad=True)
tensor([-0.1917])
True


In [7]:
# 一次性访问所有参数
# 遍历第一个子模块的所有可学习参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
# 遍历所有可学习参数
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [8]:
# 访问网络参数的其他方式
net.state_dict()['2.bias'].data

tensor([-0.1917])

In [9]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())


def block2():
    net = nn.Sequential()
    # 开始嵌套
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net


rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
print(rgnet(X))

tensor([[0.1195],
        [0.1195]], grad_fn=<AddmmBackward0>)


In [10]:
# 查看该嵌套块是如何工作的
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [11]:
# 访问第一个块中，第二个子块，第一层的偏置项
print(rgnet[0][1][0].bias)

Parameter containing:
tensor([-0.4909, -0.3902, -0.2360, -0.2115,  0.1225, -0.3940, -0.2918,  0.0433],
       requires_grad=True)


In [12]:
# 内置初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)  # 将权重初始化为标准差 0.01的高斯随机变量
        nn.init.zeros_(m.bias)  # 将偏差初始化为零


net.apply(init_normal)
print(net[0].weight.data[0], net[0].bias.data[0])

tensor([-0.0118,  0.0106,  0.0060,  0.0036]) tensor(0.)


In [13]:
# 初始化为给定常数
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)  # 将权重初始化为1
        nn.init.zeros_(m.bias)


net.apply(init_constant)
print(net[0].weight.data[0], net[0].bias.data[0])

tensor([1., 1., 1., 1.]) tensor(0.)


In [14]:
# 采用 Xavier 初始化第一个神经网络层
# 将第三个神经网络层初始化为 42
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)


def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)


net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.2979,  0.4898, -0.5295, -0.4871])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [17]:
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        # 保留绝对值大于5的权重，其余权重设置为0
        m.weight.data *= m.weight.data.abs() > 5


net.apply(my_init)
print(net[0].weight.data[0])

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])
tensor([-9.1541,  0.0000,  0.0000, -9.6616])


In [18]:
# 我们可以始终直接设置参数
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000, -8.6616])

In [22]:
# 参数绑定/共享参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(), shared,
                    nn.ReLU(), nn.Linear(8, 1))
print(net(X))
# 检查参数是否相同
print(net[2].weight.data[0] ==  net[4].weight.data[0])
net[2].weight.data[0,0] = 100
# 确保对象是同一个，而不仅仅是一个值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([[0.2342],
        [0.2371]], grad_fn=<AddmmBackward0>)
tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
